In [ ]:
%load_ext autoreload
%autoreload 2cd 
%config InlineBackend.figure_format = 'retina'

import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
lr = 0.001
epochs = 10
batch_size = 2048
hidden_units = 500
max_features = 15000
_max_df = 0.11
_min_df = 1

Preguntas teóricas:
1) Qué relación hay entre el TF-IDF y el CountVectorizer?
Ambos son medios de conversión de palabras a números para que una computadora pueda asignarles algún tipo de peso. Count Vectorizer se apoya en la frecuencia de aparición de las palabras, mientras que TF-IDF (Term Frequency - Inverse Document Frequency) usa una expresión más refinada que le da más peso a una palabra si aparece muchas veces en un mismo documento pero se lo reduce si aparece en muchos documentos distintos, ayudando a reducir la dimensionalidad de entrada al permitirnos filtrar términos menos importantes. La idea es que palabras que aparecen demasiadas veces o demasiado poco, tienen menor relevancia estadística.

2) Explique el concepto de Bag Of Words
Es un modelo que vectoriza un texto a partir de las palabras que aparecen en él, y la cantidad de veces que aparecen. No preserva orden, ya que sólo le importa la cantidad de ocurrencias (o multiplicidad).  


3) Para validar sus resultados deberá utilizar siempre el método Hold-Out validation. Explíquelo y compárelo con K-Folding (investigar).
Hold out validation consiste en retener (o holdear) una parte del set de datos destinado al entrenamiento, la cual, una vez entrenado el modelo, se va a utilizar para corroborar el performance del mismo. K-folding es una idea similar, con la salvedad de repite el proceso múltiples veces y en cada iteración cambia el subconjunto de pruebas. K-folding es especialmente útil en los casos donde el set de muestras es limitado, aunque requiere más poder de cómputo.

4) Explique la métrica utilizada en la competencia (Macro AUC-ROC). Investigue la métrica F1-score. Compárelas. 
AUC-ROC (Area under the curve - Receiver Operation Characteristic) es la composición de dos conceptos separados. En primer lugar, ROC es una métrica para la clasificación binaria, basada en comparar la tasa de verdaderos positivos contra falsos positivos. Se produce una curva al variar parámetros de nuestro modelo, en el cual al aumentar los verdaderos positivos, eventualmente también aumentan los falsos positivos. Un modelo bueno aumentará sus verdaderos positivos sin aumentar significativamente sus falsos positivos. La curva resultante se le aplica el AUC, Area Under Curve. Si el area debajo de la curva formada en el ROC es 1, el modelo es ideal e identifica correctamente todos los positivos sin incurrir en falsos positivos adicionales. Si no, igualmente se puede considerar funcional mientras el área sea mayor que 0.5, a partir de donde el modelo estaría prediciendo de forma prácticamente aleatoria. Un área menor a 0.5 implica que el modelo está adivinando de forma inversa a la que debería.

# Importo dataset

In [ ]:
folder = './'
train = pd.read_csv('../input/toxic-comments-challenge/train.csv')
test = pd.read_csv('../input/toxic-comments-challenge/test.csv')
submission = pd.read_csv('../input/toxic-comments-challenge/sample_submission.csv')

import shutil

original = r'/kaggle/input/porfavor/helper.py' #Esto es para evitar cargar los helpers a mano cada vez.
target = r'/kaggle/working/helper.py'
shutil.copyfile(original, target)

original = r'/kaggle/input/porfavor/toxic_helper.py'
target = r'/kaggle/working/toxic_helper.py'
shutil.copyfile(original, target)

original = r'/kaggle/input/porfavor/multiclass_helper.py'
target = r'/kaggle/working/multiclass_helper.py'
shutil.copyfile(original, target)

original = r'/kaggle/input/porfavor/fnn_helper.py'
target = r'/kaggle/working/fnn_helper.py'
shutil.copyfile(original, target)

In [ ]:
#train[:10] #Solo para ver contenido del archivo de train, cargado con pandas.
train.head() #Muestra los datos de train que estoy usando, lo mismo que arriba pero con .head() de pandas.

In [ ]:
train.shape #Me devuelve el tamaño en columnas y filas del archivo.

In [ ]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values #Devuelve los valores de las columnas incluidas en list_classes
#y[:10] #Hasta aca nada muy loco. Es funciones varias de Python y pandas.

# Divido entre train y valid

In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(train, y, test_size = 0.1) #Split arrays or matrices into random train and test subsets.

#test_sizefloat or int, default=None
#If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the test split. 
#If int, represents the absolute number of test samples. If None, the value is set to the complement of the train size. If train_size is also None, it will be set to 0.25.

# Analisis rapido

In [ ]:
toxic_ratio = (Y_train.sum(axis = 1) > 0).sum()/Y_train.shape[0] #Shape devuelve (filas, columnas). sum(axis = 1) colapsa las columnas en una sola sumando sus contenidos.
print(toxic_ratio) #Este código básicamente se fija cuales comentarios tienen algún tipo de toxicidad y luego calcula el porcentaje de comentarios tóxicos frente al total.

In [ ]:
0.96-toxic_ratio + toxic_ratio*0.873 # Ni idea.

In [ ]:
X_train.shape #Puedo ver que de los 159k comentarios tomamos solo 143k para entrenar el modelo.

In [ ]:
raw_text_train = X_train["comment_text"].str.lower() #Paso todo a lowercase, para independizar al analisis de las mayúsculas.
raw_text_valid = X_valid["comment_text"].str.lower()
raw_text_test = test["comment_text"].str.lower()

In [ ]:
print(raw_text_train[0:10]) # Recordar que train_test_split hace shuffle 

In [ ]:
Y_train[:10]

# Stemming o Lemmatizer

Stemming usa la raíz de la palabra para clasificarla, considerando a cualquier variación de una palabra como la misma palabra.
Lemmatizer utiliza el contexto de la palabra.
"Regular dictionaries are lists of lemmas, not stems"
Problema de stem: Dos palabras distintas pueden tener el mismo stem.


In [ ]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
class LemmaTokenizer(object): #Creo una clase lista para procesar con lemmatizer.
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]
    
lemma = LemmaTokenizer()

In [ ]:
from nltk.stem.porter import PorterStemmer
import re
token_pattern=r"(?u)\b\w\w+\b"
compiled_reg_exp = re.compile(token_pattern)
def tokenize(text): #Defino una función para recuperar el stem de las palabras.
    tokens = compiled_reg_exp.findall(text)
    stems = []
    for item in tokens:
        if len(item)>100:
            item = 'tooLongWord'
        stems.append(PorterStemmer().stem(item))
    return stems

# Armo matriz de features

Features in a neural network are the variables or attributes in your data set. You usually pick a subset of variables that can be used as good predictors by your model. So in a neural network, the features would be the input layer, not the hidden layer nodes. The output is whatever variable (or variables) you're trying to predict.


max_dffloat or int, default=1.0
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float in range [0.0, 1.0], the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

min_dffloat or int, default=1
When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float in range of [0.0, 1.0], the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer #TfidfVectorizer - Transforms text to feature vectors that can be used as input to estimator.

#max_features = 10000

tfidf_vectorizer = TfidfVectorizer(max_df = _max_df, min_df = _min_df,
                                   max_features=max_features,
                                   #tokenizer=tokenize,
                                   stop_words='english')

%time tfidf_matrix_train = tfidf_vectorizer.fit_transform(raw_text_train) #Procesa la X_Train (raw text train) y lo transforma en una matriz esparsa 

Regarding tfidf_vectorizer.fit_transform: https://stackoverflow.com/questions/50906210/confused-with-the-return-result-of-tfidfvectorizer-fit-transform

You are printing a sparse matrix so the output looks different compared to printing a standard dense matrix. See below the main components:

The tuple represents: (document_id, token_id)

The value following the tuple represents the tf-idf score of a given token in a given document

The tuples that are not there have a tf-idf score of 0

If you want to find what token the token_id corresponds to, check the get_feature_names method.

In [ ]:
%time tfidf_matrix_valid = tfidf_vectorizer.transform(raw_text_valid) #Hago lo mismo con la validación, pero con transform en lugar de fit_transform. 

In [ ]:
%time tfidf_matrix_test = tfidf_vectorizer.transform(raw_text_test) #Voy a tener que preguntar cual es la diferencia.

## Sparsity

In [ ]:
sparsity = 1 - (tfidf_matrix_train>0).sum()/(tfidf_matrix_train.shape[0]*tfidf_matrix_train.shape[1]) #La cantidad de palabras que cumplen los criterios de maxdf y mindf dividido las palabras totales.
print(sparsity)

## TFIDF Results

In [ ]:
tfidf_matrix_train.shape

sin lemma
- 177719 con 0.95, 1
- 177712 con 0.11, 1


In [ ]:
top_10 = np.argsort(tfidf_matrix_train.sum(axis=0))[0,::-1][0,:10].tolist()[0] #Lol?
feature_names = np.array(tfidf_vectorizer.get_feature_names()) #Devuelve el nombre de los features y los vectoriza, luego los hace arreglo?
print(feature_names[np.array(top_10)]) #Imprime las 10 palabras que más aparecen en los features.

## Sin reducción de dimensionalidad

In [ ]:
#dense_matrix_train = tfidf_matrix_train.todense()

In [ ]:
#dense_matrix_train.shape, Y_train.shape

In [ ]:
#dense_matrix_valid = tfidf_matrix_valid.todense()

## Reducimos dimensionalidad

In [ ]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
trunSVD = TruncatedSVD(n_components=300)
%time dense_matrix_train = trunSVD.fit_transform(tfidf_matrix_train)

In [ ]:
%time dense_matrix_valid = trunSVD.transform(tfidf_matrix_valid)

In [ ]:
dense_matrix_train.shape, dense_matrix_valid.shape

In [ ]:
%time dense_matrix_test = trunSVD.transform(tfidf_matrix_test)

# Modelo de 1 capa densa

In [ ]:
#!pip install toxic_helper
!pip install detoxify
from toxic_helper import auc

In [ ]:
from tensorflow import keras as keras
from keras.models import Sequential
from keras import optimizers
from keras.layers.core import Dense, Activation
from helper import PlotLosses
from keras.callbacks import ModelCheckpoint
from keras.initializers import RandomNormal
from keras import regularizers
from keras import initializers

In [ ]:
default_initializer = initializers.normal(mean=0, stddev=0.001) #Defino una gaussiana por algún motivo u otro.
# default_initializer = 'zeros'

In [ ]:
input_features = dense_matrix_train.shape[1]  #Cantidad de features de entrada
output_size = Y_train.shape[1] #Dimensión de la salida
#hidden_units = 100
lambd = 0 #0.001
model_sig_nn = Sequential() #Es el modelo. Le puedo agregar las capas que quiera.
model_sig_nn.add(Dense(hidden_units,
                       input_dim=input_features, 
                       kernel_regularizer=regularizers.l2(lambd), 
                       kernel_initializer=default_initializer,
                       name="Capa_Oculta_1"))  # https://keras.io/api/layers/core_layers/dense/
model_sig_nn.add(Activation('sigmoid')) #Capas con activación sigmoidea, 50 neuronas, input_features entradas, y tengo que ver qué onda los regularizers y el initializer.
model_sig_nn.add(Dense(hidden_units,
                       input_dim=input_features, 
                       kernel_regularizer=regularizers.l2(lambd), 
                       kernel_initializer=default_initializer,
                       name="Capa_Oculta_2"))
model_sig_nn.add(Activation('sigmoid'))
model_sig_nn.add(Dense(output_size,
                       kernel_regularizer=regularizers.l2(lambd), 
                       kernel_initializer=default_initializer,
                       name="Capa_Salida"))
model_sig_nn.add(Activation('sigmoid', name="output")) 
model_sig_nn.summary()


#lr = 0.1
#batch_size = 64
#epochs = 10

#selectedOptimizer = keras.optimizers.SGD(learning_rate=lr)
selectedOptimizer = keras.optimizers.Adam(lr=lr)

model_sig_nn.compile(loss = 'binary_crossentropy', optimizer=selectedOptimizer, 
                     metrics=['accuracy']) #auc

In [ ]:
model_sig_nn.evaluate(dense_matrix_valid, Y_valid)

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='/kaggle/working/basic_model_best.hdf5', verbose=1, save_best_only=True) #verbose=1 will show you an animated progress bar

plot_losses = PlotLosses(plot_interval=1, 
                         evaluate_interval=5, 
                         x_val=dense_matrix_valid, 
                         y_val_categorical=Y_valid)
history = model_sig_nn.fit(dense_matrix_train, 
          Y_train, 
          batch_size = batch_size,
          epochs=epochs, 
          verbose=1, 
          validation_data=(dense_matrix_valid, Y_valid), 
          callbacks=[plot_losses, checkpointer],
         )

# Evaluo valid

In [ ]:
model_sig_nn.load_weights('/kaggle/working/basic_model_best.hdf5')

In [ ]:
tfidf_matrix_valid.shape, Y_valid.shape

In [ ]:
pred_valid = model_sig_nn.predict(dense_matrix_valid, verbose = 1)
pred_train = model_sig_nn.predict(dense_matrix_train, verbose = 1)
pred_test = model_sig_nn.predict(dense_matrix_test, verbose = 1)

In [ ]:
model_sig_nn.evaluate(dense_matrix_valid, Y_valid)

[0.073181636426992352, 0.97782721309109666]

# ROC Curve

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from scipy import interp
from itertools import cycle

print(roc_auc_score(Y_train, pred_train, average='macro'))
print(roc_auc_score(Y_valid, pred_valid, average='macro'))

0.930758815168  
0.920441587397

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()
n_classes = Y_valid.shape[1]
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(Y_valid[:, i], pred_valid[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    
fpr["micro"], tpr["micro"], _ = roc_curve(Y_valid.ravel(), pred_valid.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
from matplotlib import pyplot as plt
# Compute macro-average ROC curve and ROC area
lw = 2
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

http://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html

**True Positive Rate**:   
The number of times your system was able to classify the positives as positives. 

True positive rate = Correctly Classified Positives/(Correctly Classified as Positives+ Falsely Classified as Negatives)

**False Positive Rate**:  
The number of times your system classified a negative as a positive divided by the total  actual negative instances.


False positive rate = Incorrectly Classified as Positives/(Incorrectly Classified as Positives+ Correctly classified as Negatives )

https://en.wikipedia.org/wiki/Receiver_operating_characteristic

https://datascience.stackexchange.com/questions/15989/micro-average-vs-macro-average-performance-in-a-multiclass-classification-settin/16001

Macro-average: Calcula el score de cada clase y luego promedia  
Micro-average: Suma y luego calcula el score

Micro-average se considera mejor cuando hay desbalce en las clases

# Interpretación

- toxic
- severe_toxic
- obscene
- threat
- insult
- identity_hate

In [ ]:
(model_sig_nn.get_weights()[0]).shape

In [ ]:
salida = 2
sorted_indexes = np.argsort(model_sig_nn.get_weights()[0][:,salida])[::-1]
np.array(tfidf_vectorizer.get_feature_names())[sorted_indexes][:20]

# Predict for test

In [ ]:
tfidf_matrix_test = tfidf_vectorizer.transform(raw_text_test)

In [ ]:
#dense_matrix_test = tfidf_matrix_test.todense()

In [ ]:
pred = model_sig_nn.predict(dense_matrix_test, verbose=1)

In [ ]:
pred.shape

In [ ]:
1*(pred[0:10]>0.5)

In [ ]:
submission[list_classes] = pred_test
submission.to_csv("submission_TSVD_300.csv", index = False)

Submit1 ROC AUC en valid = 0.731196488262  
Submit2 ROC AUC en valid = 0.963410980044  
Submit3 ROC AUC en valid = 0.974042855266

In [ ]:
# ~/.local/bin/kaggle competitions submit -c jigsaw-toxic-comment-classification-challenge -f submission_early_stop_2_epochs.csv -m "Early stop 2 epochs"